In [54]:
import pandas as pd
import os
import numpy as np
import optuna
import logging
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError
import json

entorno = 'VM'  # Elegir "VM" o "local" para correr en entorno local
nombre_experimento = 'LSTM_producto_FE_optuna'
ventana_input = 12
ventana_output = 2
ventana_test = 3
lags = 6
cant_productos_considerar = 50
num_trials = 30



# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = os.path.expanduser('~/buckets/b1/datasets')
    carpeta_exp_base = os.path.expanduser('~/buckets/b1/exp')
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)

    
# Configurar logger de Optuna
optuna.logging.set_verbosity(optuna.logging.INFO)
logger = optuna.logging.get_logger("optuna")
log_path = os.path.join(carpeta_exp, 'optuna_log.txt')
handler = logging.FileHandler(log_path)
handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)  

dataset_completo = pd.read_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'))


dataset_completo.head()

,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,5,2,0,4,7.79714,0.50130,0.09348,0.00000


In [55]:
ventas_producto_mes = dataset_completo.groupby(['Timestamp', 'product_id', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion']).agg({
    'tn': 'sum',
    'edad_producto': 'first'
})
ventas_producto_mes = ventas_producto_mes.reset_index()

#Generar ventas por categoria y agregar al dataset
ventas_cat1 = ventas_producto_mes.groupby(['Timestamp', 'cat1'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat1'})
ventas_cat2 = ventas_producto_mes.groupby(['Timestamp', 'cat1', 'cat2'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat2'})
ventas_cat3 = ventas_producto_mes.groupby(['Timestamp', 'cat1', 'cat2', 'cat3'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_cat3'})
ventas_familia_productos = ventas_producto_mes.groupby(['Timestamp', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion'])['tn'].sum().reset_index().rename(columns={'tn': 'ventas_familia_producto'})

ventas_producto_mes = ventas_producto_mes.merge(ventas_cat1, how='left', on=['Timestamp','cat1'])
ventas_producto_mes = ventas_producto_mes.merge(ventas_cat2, how='left', on=['Timestamp', 'cat1', 'cat2'])
ventas_producto_mes = ventas_producto_mes.merge(ventas_cat3, how='left', on=['Timestamp', 'cat1', 'cat2', 'cat3'])
ventas_producto_mes = ventas_producto_mes.merge(ventas_familia_productos, how='left', on=['Timestamp', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion'])

ventas_producto_mes.drop(columns=['cat1', 'cat2', 'cat3', 'brand', 'descripcion'], inplace = True)
ventas_producto_mes.set_index('Timestamp', inplace=True)
ventas_producto_mes

,product_id,tn,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
Timestamp,,,,,,,
2017-01-01,20001,934.77222,0,16043.73199,8020.78937,2865.69383,1165.80692
2017-01-01,20002,550.15707,0,16043.73199,8020.78937,2865.69383,789.91542
2017-01-01,20003,1063.45835,0,5630.46989,4538.52058,3617.40066,2546.73899
2017-01-01,20004,555.91614,0,5630.46989,4538.52058,3617.40066,2546.73899
2017-01-01,20005,494.27011,0,5630.46989,4538.52058,3617.40066,2546.73899
...,...,...,...,...,...,...,...
2019-12-01,21263,0.01270,14,5106.38727,3036.02058,1492.68623,0.01270
2019-12-01,21265,0.05007,9,5106.38727,81.96802,0.27021,0.05007
2019-12-01,21266,0.05121,9,5106.38727,81.96802,0.27021,0.05121


In [56]:
ventas_dic_2019 = ventas_producto_mes[ventas_producto_mes.index == '2019-12-01']
ventas_dic_2019 = ventas_dic_2019.sort_values(by='tn', ascending = False)
lista_productos_mayores_ventas = list(ventas_dic_2019.iloc[:cant_productos_considerar]['product_id'].values)
lista_productos_mayores_ventas

[20001, 20002, 20003, 20004, 20005]

In [57]:
def crear_dataset_supervisado(array, input_length, output_length):
    # Inicialización
    X, Y = [], []    # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape) == 1:  # Si tenemos sólo una serie (univariado)
        array = array.reshape(-1, 1)
        cols = 1
    else:  # Multivariado
        fils, cols = array.shape

    # Generar los arreglos (utilizando ventanas deslizantes de longitud input_length)
    for i in range(fils - input_length - output_length + 1):
        X.append(array[i:i + input_length, :].reshape(input_length, cols))
        Y.append(array[i + input_length:i + input_length + output_length, -1].reshape(output_length, 1))

    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

In [58]:
# Función para crear el modelo LSTM
def crear_modelo_lstm(input_shape, units_lstm, dropout_rate, learning_rate):
    model = Sequential()
    model.add(LSTM(units_lstm[0], return_sequences=True, input_shape=input_shape, recurrent_dropout=0.25))
    model.add(LSTM(units_lstm[1], return_sequences=True, recurrent_dropout=0.25))
    model.add(LSTM(units_lstm[2], recurrent_dropout=0.25))
    model.add(Dropout(dropout_rate))
    model.add(Dense(ventana_output))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate), metrics=[MeanSquaredError()])
    return model

In [59]:
# Función para la optimización con Optuna
def objective(trial):
    # Definir hiperparámetros a optimizar
    units_lstm = [
        trial.suggest_int('units_lstm_1', 32, 128),
        trial.suggest_int('units_lstm_2', 32, 128),
        trial.suggest_int('units_lstm_3', 32, 128)
    ]
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_int('batch_size', 1, 32)
    epochs = trial.suggest_int('epochs', 50, 200)

    lista_productos_LSTM = []
    lista_predicciones_LSTM = []
    loss_list = []

    for producto in lista_productos_mayores_ventas:
        ventas_mes_por_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto].copy()
        ventas_mes_por_producto.drop(columns=['product_id'], inplace=True)

        # Escalar valor
        scaler_tn = RobustScaler()
        scaler_edad = RobustScaler()
        scaler_ventas_cat1 = RobustScaler()
        scaler_ventas_cat2 = RobustScaler()
        scaler_ventas_cat3 = RobustScaler()
        ventas_familia_producto = RobustScaler()
        ventas_mes_por_producto['tn'] = scaler_tn.fit_transform(np.array(ventas_mes_por_producto['tn']).reshape(-1,1))
        ventas_mes_por_producto['edad_producto'] = scaler_edad.fit_transform(np.array(ventas_mes_por_producto['edad_producto']).reshape(-1,1))
        ventas_mes_por_producto['ventas_cat1'] = scaler_ventas_cat1.fit_transform(np.array(ventas_mes_por_producto['ventas_cat1']).reshape(-1,1))
        ventas_mes_por_producto['ventas_cat2'] = scaler_ventas_cat2.fit_transform(np.array(ventas_mes_por_producto['ventas_cat2']).reshape(-1,1))
        ventas_mes_por_producto['ventas_cat3'] = scaler_ventas_cat3.fit_transform(np.array(ventas_mes_por_producto['ventas_cat3']).reshape(-1,1))
        ventas_mes_por_producto['ventas_familia_producto'] = ventas_familia_producto.fit_transform(np.array(ventas_mes_por_producto['ventas_familia_producto']).reshape(-1,1))

        
        


        if len(ventas_mes_por_producto) >= (ventana_input + ventana_output):
            # Formatear valores para input LSTM
            X, Y = crear_dataset_supervisado(ventas_mes_por_producto.values, ventana_input, ventana_output)

            # Separar datos en entrenamiento y test
            train_size = len(X) - ventana_test
            X_train, X_test = X[:train_size], X[train_size:]
            Y_train, Y_test = Y[:train_size], Y[train_size:]

            # Crear y ajustar el modelo LSTM
            model = crear_modelo_lstm((ventana_input, X.shape[2]), units_lstm, dropout_rate, learning_rate)
            model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            # Evaluar el modelo
            loss = model.evaluate(X_test, Y_test, verbose=0)
            loss_list.append(loss)

    return np.mean(loss_list)

In [60]:
# Ejecución de Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=num_trials)

# Guardar los mejores hiperparámetros
best_params = study.best_params
with open(os.path.join(carpeta_exp, 'mejores_hiperparametros.json'), 'w') as f:
    json.dump(best_params, f)

print("Mejores hiperparámetros encontrados: ", best_params)

[I 2024-07-12 19:04:02,127] A new study created in memory with name: no-name-0019659b-9ef5-4985-a8d7-80db115d12f7


[I 2024-07-12 19:04:45,714] Trial 0 finished with value: 3.1907825738191606 and parameters: {'units_lstm_1': 75, 'units_lstm_2': 59, 'units_lstm_3': 110, 'dropout_rate': 0.3445095500515053, 'learning_rate': 0.007283594830097051, 'batch_size': 12, 'epochs': 131}. Best is trial 0 with value: 3.1907825738191606.


[I 2024-07-12 19:07:44,136] Trial 1 finished with value: 3.0639173805713655 and parameters: {'units_lstm_1': 114, 'units_lstm_2': 114, 'units_lstm_3': 46, 'dropout_rate': 0.28857803585302594, 'learning_rate': 0.0083483944530117, 'batch_size': 6, 'epochs': 126}. Best is trial 1 with value: 3.0639173805713655.


Mejores hiperparámetros encontrados:  {'units_lstm_1': 114, 'units_lstm_2': 114, 'units_lstm_3': 46, 'dropout_rate': 0.28857803585302594, 'learning_rate': 0.0083483944530117, 'batch_size': 6, 'epochs': 126}
